In [9]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import os
import cv2
import imutils
from imutils.video import VideoStream

In [4]:
def detect_and_predict(frame,faceNet,maskNet):
    (h,w)=frame.shape[:2]
    blob=cv2.dnn.blobFromImage(frame,1.0,(300,300),(104.0,177.0,123.0))
    faceNet.setInput(blob)
    detections=faceNet.forward()
    
    #initialize list of faces in frames,corresponding locations,and list of predictions
    faces=[]
    locs=[]
    preds=[]
    
    #loop over detections
    for i in range(0,detections.shape[2]):
        confidence=detections[0,0,i,2]
    #     print(confidence)
        if(confidence>0.5):
            box=detections[0,0,i,3:7]*np.array([w,h,w,h])
            (startX,startY,endX,endY)=box.astype('int')

            #ensure that the dimensions of box are under the frame window
            (startX,startY)=(max(0,startX),max(0,startY))
            (endX,endY)=(min(w-1,endX),min(h-1,endY))
            
            face=frame[startY:endY,startX:endX]
            face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
            face=cv2.resize(face,(224,224))
            face=img_to_array(face)
            face=preprocess_input(face)
            
            faces.append(face)
            locs.append((startX,startY,endX,endY))
            
        #if there is face then only make prediction
        if len(faces)>0:
            faces=np.array(faces,dtype='float32')
            preds=maskNet.predict(faces,batch_size=12)
        return(locs,preds)

In [5]:
prototxtPath=os.path.sep.join([r'H:\face mask detector\face_detector','deploy.prototxt'])
wieghtsPath=os.path.sep.join([r'H:\face mask detector\face_detector','res10_300x300_ssd_iter_140000.caffemodel'])

In [6]:
faceNet=cv2.dnn.readNet(prototxtPath,wieghtsPath)

In [7]:
maskNet=load_model(r'H:\face mask detector\model.h5')

In [10]:
vs=cv2.VideoCapture(0)
# vs.start()

while True:
    #grab the from threaded video stream and resize it 
    #to have max width 400 px
#     frame=vs.read()
    ret,frame=vs.read()
    
    (locs,preds)=detect_and_predict(frame,faceNet,maskNet)
    
    #loop over the detected face locations and display box over them
    
    for(box,pred) in zip(locs,preds):
        (startX,startY,endX,endY)=box
        (mask,withoutMask)=pred
        
        #determine the class label and color accordingly
        label='Mask' if (mask>withoutMask) else 'No Mask'
        color=(0,255,0) if label=='Mask' else (0,0,255)
        
        #show the probability of the mask
        label="{}:{:.2f}%".format(label,max(mask,withoutMask)*100)
        
        
        cv2.putText(frame,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(frame,(startX,startY),(endX,endY),color,3)
    
    cv2.imshow('frame',frame)
    key=cv2.waitKey(1)& 0xff
    
    if key==ord('q'):
        break
        
cv2.destroyAllWindows()
vs.release()
        